In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 12.6 MB/s eta 0:00:00


In [ ]:
! conda install -c conda-forge spacy
! python -m spacy download it_core_news_sm

/bin/bash: line 1: conda: command not found
2024-01-28 15:03:15.365883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 15:03:15.365939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 15:03:15.367299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 15:03:15.374810: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-2

In [ ]:
import re
import nltk
import emoji
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from itertools import groupby
import spacy
nlp = spacy.load("it_core_news_sm")
import pandas as pd
import string

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Data Preprocessing


In [ ]:
correct_words = ['alcool', 'cool', 'preesame', ]

In [ ]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    #print(text)
    text = re.sub(r' aff[xx]+lo ', ' affanculo ', text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text)
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text)
    text = re.sub(r' inc[az.x*@%#$^]+to ', ' incazzato ', text)
    text = re.sub(r' c[az.x*@%#$^]+te ', ' cazzate ', text)
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text)
    text = re.sub(r' caz[x]', 'cazzate', text)
    text = re.sub(r' c[az.x*@%#$^]+one ', ' cazzone ', text)
    text = re.sub(r' m[erd.x*@%#$^]+a ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text)
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text)
    text = re.sub(r' p[tu.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]
        vocali = ['a','e','i','o','u','y']

        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0]
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in correct_words):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
    return(stringa)

def replace_verbs_with_root(sentence):
     # Dictionary to handle special cases manually
    special_cases = {
        "c'è": "essere",
        "c'erano": "essere",
        "c'era": "essere",
        "ci sono": "essere"
    }

    # Replace special cases in the sentence
    for word, root in special_cases.items():
        sentence = sentence.replace(word, root)

    return sentence

def preprocess_tweet(tweet):
    # Remove newlines and carriage returns and lowercase words
    tweet = tweet.replace('\n', ' ').replace('\r', ' ').lower()

    # Remove incorrect bad word
    tweet = clean_disguised_bad_words(tweet)

    # Remove duplicates
    tweet = deleteDuplicate(tweet)

    # Substitute root verbs
    tweet = replace_verbs_with_root(tweet)

    # Substitute URLs with <URL>
    tweet = re.sub(r'\[url\]', '<URL>', tweet)

    # Substitute mentions with <MENTION>
    tweet = re.sub(r'@\w+', '<MENTION>', tweet)

    # Substitute hashtags with <HASHTAG>
    tweet = re.sub(r'#\w+', '<HASHTAG>', tweet)

    # Replace emojis with text
    tweet = emoji.demojize(tweet, language='it')

    # Tokenization
    tokens = nltk.word_tokenize(tweet)

    # Remove stop words (Italian)
    stop_words = set(stopwords.words('italian'))
    filtered_tokens = " ".join(word for word in tokens if word not in stop_words)

    """ # Stemming using Italian stemmer
    stemmer = SnowballStemmer('italian')
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens] """

    # Lemmatize using spaCy
    doc = nlp(filtered_tokens)
    lemmatize_tokens = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]

    # Remove special characters and numbers, preserve placeholders
    processed_tweet = " ".join(re.sub(r'[^a-zA-Z\<\>]', ' ', token) for token in lemmatize_tokens)

    # Use re.sub() to replace spaces within "< >"
    result = re.sub(r"<\s*(\w+)\s*>", r"<\1>", processed_tweet)

    return result

### Preprocessing alternativa

In [ ]:
# pre processing alternativa

def clean_disguised_bad_words(text):
    text = " " + text + " "
    #print(text) #possibili altre parolacce trovate: "fankullllllll!", stron**ta, stro...🤐🤐
    text = re.sub(r' aff[xx]+lo ', ' affanculo ', text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text)
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text)
    text = re.sub(r' inc[az.x*@%#$^]+to ', ' incazzato ', text)
    text = re.sub(r' c[az.x*@%#$^]+te ', ' cazzate ', text)
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text)
    text = re.sub(r' caz[x]', 'cazzate', text)
    text = re.sub(r' c[az.x*@%#$^]+one ', ' cazzone ', text)
    text = re.sub(r' m[erd.x*@%#$^]+a ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text)
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text)
    text = re.sub(r' p[tu.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]
        vocali = ['a','e','i','o','u','y']

        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0]
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in correct_words):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
    return(stringa)

def replace_verbs_with_root(sentence):
     # Dictionary to handle special cases manually
    special_cases = {
        "c'è": "essere",
        "c'erano": "essere",
        "c'era": "essere",
        "ci sono": "essere"
    }

    # Replace special cases in the sentence
    for word, root in special_cases.items():
        sentence = sentence.replace(word, root)

    return sentence

def preprocess_tweet(tweet):
    # Remove newlines and carriage returns and lowercase words
    tweet = tweet.replace('\n', ' ').replace('\r', ' ')

    # Remove incorrect bad word
    tweet = clean_disguised_bad_words(tweet)

    # Remove duplicates
    tweet = deleteDuplicate(tweet)

    # Substitute root verbs
    tweet = replace_verbs_with_root(tweet)

    # Substitute URLs with <URL>
    tweet = re.sub(r'\[url\]', '<URL>', tweet)

    # Substitute mentions with <USER>
    tweet = re.sub(r'@(\w+)', '<USER>', tweet)

    # Substitute hashtags with the word
    tweet = re.sub(r'#(\w+)', lambda match: match.group(1).lower(), tweet)

    # Replace emojis with text
    tweet = emoji.demojize(tweet, language='it')

    # Tokenization
    tokens = nltk.word_tokenize(tweet)

    # Remove stop words (Italian)
    #stop_words = set(stopwords.words('italian'))
    #filtered_tokens = " ".join(word for word in tokens if word not in stop_words)
    #filtered_tokens = " ".join(word for word in tokens)

    tokens = " ".join(word for word in tokens)

    """ # Stemming using Italian stemmer
    stemmer = SnowballStemmer('italian')
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens] """

    # Lemmatize using spaCy
    doc = nlp(tokens)
    lemmatize_tokens = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]

    # Remove special characters and numbers, preserve placeholders
    #processed_tweet = " ".join(re.sub(r'[^a-zA-Z\<\>]', ' ', token) for token in lemmatize_tokens)

    # Remove punctuation
    processed_tweet = " ".join(token.strip(string.punctuation) for token in lemmatize_tokens)

    # Use re.sub() to replace spaces within "< >"
    result = re.sub(r"<\s*(\w+)\s*>", r"<\1>", processed_tweet)

    return result


Per fare delle prove usa la cella di sotto

In [ ]:

# Example Italian tweet with placeholders
tweet = """Ma davveroooooooo 📺 #SalviniMerda49Milioni
  alcool c'è che l'ha 1 ciaaaooo c@lo ca**o m*rd@ una
  raccolta FIRME per #NoCoprifuocoDeiMieiCoglioni riuscirà a far cambiare idea al #governodeipeggiori?
  Povero illuso del cazzo @12341234 [URL]"""

# Preprocess the tweet
processed_tweet = preprocess_tweet(tweet)

print("Processed Tweet:", processed_tweet)

 Ma davveroooooooo 📺 #SalviniMerda49Milioni   alcool c'è che l'ha 1 ciaaaooo c@lo ca**o m*rd@ una   raccolta FIRME per #NoCoprifuocoDeiMieiCoglioni riuscirà a far cambiare idea al #governodeipeggiori?   Povero illuso del cazzo @12341234 [URL] 
Processed Tweet: ma davvero  televisore  salvinimerda49milione alcool essere che lo avere 1 ciao culo cazzo merdare uno raccolta FIRME per nocoprifuocodeimieicoglione riuscire a fare cambiare idea a il governodeipeggiore  povero illudere di il cazzo  USER   URL 


### Applico la funzione di data preprocessing a training_textual

In [ ]:
# Specify the path to your CSV file
csv_file = '/content/drive/Shareddrives/HLT/dataset/development/training_textual.csv'
dev_data = pd.read_csv(csv_file)
new_dev_data = dev_data.copy()
new_dev_data['anonymized_text'] = new_dev_data['anonymized_text'].apply(preprocess_tweet)

Output streaming troncato alle ultime 5000 righe.
 La #legge umana intanto è tale in quanto è conforme alla retta #ragione e quindi deriva dalla legge eterna.Quando invece una  legge è in contrasto con la ragione, la si denomina #leggeiniqua;  in tal caso però cessa di essere legge e diviene piuttosto un atto di #violenza 
 Esposito in 50 minuti ha fatto più sponde ed appoggi corretti di Icardi nei suoi 5 anni di Inter.  #icardi #inter #ICC2019 #JuveInter #esposito #marotta #conte #ausilio 
 Vi sono dei simpatici barzellettieri che spiegano come la #TAV sia la più grossa sconfitta del #m5s. È vero il contrario: è il suo più grande successo. La tattica è stata inventata da Gianroberto Casaleggio ed è sempre la stessa da più di dieci anni. 1/n 
 #Milano sgominata banda di 11 criminali che andavano in giro con la divisa della #polizia Il dodicesimo sembra che l'abbia fatta franca. Note queste caratteristiche: sul metro e ottanta, 45 anni, barbetta, accento milanese e dice spesso: "Prima g

In [ ]:
new_dev_data.head()

,anonymized_tweet_id,anonymized_text,label,dataset
0,217874450618134,USER con tutto il denuncia che si stare becc...,1,training_politics
1,360042217507605,prescrizione il t re mago avere trovare l ’...,0,training_politics
2,817917484817935,il m5s avere votare contro il unionicivile add...,1,training_politics
3,172580609652325,il lega e il m5s stare dare a bere a il opinio...,0,training_politics
4,145836038456701,che cosa cambiare questo legge caro USER co...,0,training_politics


In [ ]:
new_dev_data['label'].value_counts()

0    3456
1    2144
Name: label, dtype: int64

In [ ]:
print(dev_data['anonymized_text'][100])
print('----------------')
print(new_dev_data['anonymized_text'][100])

Piantatela brutti imbecilli razzisti.
Il vostro, e sottolineo vostro, ministro non ha difeso i confini italiani ma ha probabilmente violato gravemente il codice penale. 

#salvinilabestia #salvinidimettiti
#facciamorete #Diciotti #Senato
----------------
piantatela brutto imbecille razzista  il vostro  e sottolineare vostro  ministro non avere difendere il confine italiano ma avere probabilmente violare gravemente il codice penale  salvinilabestiare salvinidimettire facciamoreta diciotti senato


Salvo in un nuovo csv i dati processati

In [ ]:
from pathlib import Path
#filepath = Path('/content/drive/Shareddrives/HLT/dataset/development/processed/processed_training_textual.csv')
filepath = Path('/content/drive/Shareddrives/HLT/dataset/development/processed_alternativa/processed_training_textual_alternativa.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
new_dev_data.to_csv(filepath,index=False)

### Applico la funzione di data preprocessing a training_contextual, in particolare alla colonna che contiene la bio di un utente di twitter.

In [ ]:
# Specify the path to your CSV file
csv_file = '/content/drive/Shareddrives/HLT/dataset/development/training_contextual.csv'
dev_data_contextual = pd.read_csv(csv_file)
new_dev_data_contextual = dev_data_contextual.copy()
new_dev_data_contextual['anonymized_description'] = new_dev_data_contextual['anonymized_description'].apply(
    lambda x: preprocess_tweet(x) if pd.notnull(x) and x != '' else x
)

 Avete presente quegli stereotipi sui siciliani? Ecco, non credeteci: non è vero che siamo tutti simpatici... 
 Prof di latino e Deputata PD.Ama lo sport la Roma e gli [URL] vittoria non sarà nella lama che ci divide,ma nel ponte che ci porterà oltre le sfide(Gorman) 
 Medico, appassionato di Cinema d'Autore, Musica e Politica... IRASCIBILE e impossibile da etichettare: riuscirò sempre a fuggire da ogni vostra catalogazione. 
 Veneto! Assessore Regionale Sviluppo Economico, fondatore Liga Veneta, militante Lega. Autonomista impenitente. 
 Napoletano tifoso della Fiorentina e rivoluzionario pigro. Amo i selfie e odio i tramonti. 
 In prima fila sul carro #Pioli. 🔴⚫ Esperto di calcio da bar. Indinniatoh speciale. 
 mi piace correre e viaggiare; iprocrisia e furbizia mi fanno tristezza, sono socialista. 
 Esperto di migrazioni e mafie. Scrivo per Fandango Libri. Presiedo Radici Future Produzioni coop. Sono Colomba d'Oro per la Pace e ho vinto il Premio Livatino. 
 Sindaco di Pesaro - Pres

In [ ]:
new_dev_data_contextual.head()

,anonymized_tweet_id,created_at,retweet_count,favorite_count,source,is_reply,is_retweet,is_quote,anonymized_user_id,user_created_at,statuses_count,followers_count,friends_count,anonymized_description,dataset
0,217874450618134,2018-08-11,0.0,6.0,"<a href=""http://twitter.com/download/iphone"" r...",1.0,0.0,0.0,8.868893e+14,2018-04-01,554.0,748.0,753.0,Avete presente quello stereotipo su il sicilia...,training_politics
1,360042217507605,2018-11-08,154.0,448.0,"<a href=""http://twitter.com/download/iphone"" r...",0.0,0.0,0.0,2.570741e+14,2011-10-26,35043.0,10838.0,4535.0,prof di latino e Deputata PD.Ama il sport il R...,training_politics
2,817917484817935,2018-11-10,195.0,638.0,"<a href=""http://twitter.com/download/iphone"" r...",0.0,0.0,0.0,6.908473e+14,2009-09-24,139750.0,7971.0,1866.0,medico appassionato di Cinema di Autore Musi...,training_politics
3,172580609652325,2018-11-22,112.0,377.0,"<a href=""http://twitter.com/download/android"" ...",0.0,0.0,0.0,9.027930e+14,2012-11-27,3356.0,8493.0,5303.0,NaN,training_politics
4,145836038456701,2018-12-02,45.0,143.0,"<a href=""http://twitter.com/#!/download/ipad"" ...",0.0,0.0,0.0,5.308388e+14,2014-01-14,11234.0,10815.0,1810.0,Veneto Assessore Regionale Sviluppo Economico...,training_politics


In [ ]:
from pathlib import Path
#filepath = Path('/content/drive/Shareddrives/HLT/dataset/development/processed/processed_training_contextual.csv')
filepath = Path('/content/drive/Shareddrives/HLT/dataset/development/processed_alternativa/processed_training_contextual_alternativa.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
new_dev_data_contextual.to_csv(filepath,index=False)

### applico la funzione di preprocessing a test_textual_gold

In [ ]:
# Specify the path to your CSV file
csv_file = '/content/drive/Shareddrives/HLT/dataset/gold/test_textual_gold.csv'
test_textual_gold = pd.read_csv(csv_file)
new_test_textual_gold = test_textual_gold.copy()
new_test_textual_gold['anonymized_text'] = new_test_textual_gold['anonymized_text'].apply(
    lambda x: preprocess_tweet(x) if pd.notnull(x) and x != '' else x
)

 questo anno #babbonatale portato giocattolo italia a #salvini, e lui come tutti bambini diverte e poi rompe  #Commissione 
 Adesso che in mezzo alla strada grazie al vergognoso #decretoinsicurezza ci sono decine di migliaia di persone, compresi donne e bambini, che erano regolari fino a ieri, adesso vi sentite meglio?  Vi sentite più sicuri?  Se la risposta è sì non siete razzisti. Siete stronzi. 
 Circondatevi di persone che non sono diventate leghiste.  #IoNonCiSaro #salvini #Lega #Codroipo #decretosicurezza 
 Seriamente, per capire, se un ladro mi entra in casa, lo chiudo in una stanza e chiamo la polizia, è sequestro di persona, mentre se gli sparo è legittima difesa?  #decretoinsicurezza 
 Che poi è probabile che #Spataro non sia potuto andare in pensione per via della Fornero. Nel frattempo quelli del PD abbaiano ma non mordono.  Chi vuole pop corn?  #salvini #4dic #4dicembre 
 #Spataro ha usato magistralmente l'arte della retorica.  Un testo di oltre 1.000 parole per dire sempl

In [ ]:
new_test_textual_gold.head()

,anonymized_tweet_id,anonymized_text,label,dataset
0,424801448454884,questo anno babbonatale portare giocattolo Ita...,1,test_politics
1,775194088981616,adesso che in mezzo a il strada grazie a il ve...,1,test_politics
2,386928936347190,circondateve di persona che non essere diventa...,0,test_politics
3,957546674360002,seriamente per capire se uno ladro mi entrar...,0,test_politics
4,530526299555950,che poi essere probabile che spataro non esser...,0,test_politics


In [ ]:
#filepath = Path('/content/drive/Shareddrives/HLT/dataset/gold/processed/processed_test_textual_gold.csv')
filepath = Path('/content/drive/Shareddrives/HLT/dataset/gold/processed_alternativa/processed_test_textual_gold_alternativa.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
new_test_textual_gold.to_csv(filepath,index=False)